In [ ]:
%pip install --upgrade transformers pandas pycld3 tqdm sacremoses fastparquet pyarrow  

In [11]:
from transformers import pipeline
import pandas as pd 
import cld3
from tqdm import tqdm 

tqdm.pandas() 
df = pd.read_parquet('dataset.parquet')
df['lang'] = df['content'].progress_apply(lambda x : cld3.get_language(x).language)
df = df[df['lang'] == 'en']


100%|██████████| 12630/12630 [00:12<00:00, 979.18it/s] 


In [ ]:
import torch 
for target in ['ru','de','ar','zh','es','pl'] : 
    torch.cuda.empty_cache()
    t = pipeline(task='translation', model='Helsinki-NLP/opus-mt-en-' + target, device=0) 
    translated = t(df['content'].values.tolist(), batch_size=24, truncation=True ) 
    translated = [x['translation_text'] for x in translated]
    df[target+'_content'] = translated
df.to_parquet('translated.parquet')

Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You

In [ ]:
from shutil import copy 
copy('translated.parquet', '/storage/datasets/translated.parquet') 